In [44]:
import lxml.etree as ET
import pandas as pd
import sys, os

sys.path.append('../')
from cp03.merge_csv  import csv_reader, get_FileSize

fileheader = pd.read_excel("./CP4- header 规划.xlsx",index_col="文件名").to_dict("index")

fileheader[list(fileheader.keys())[0]]


def qlr_csv2xml(filename, header):
    XML_NAMESPACE = 'http://www.w3.org/2001/XMLSchema-instance'
    XML = '{%s}' % XML_NAMESPACE
    NSMAP = {None : XML_NAMESPACE}

    QlrFile = ET.Element(XML + "QlrFile", nsmap = NSMAP)
    QlrFile.set(XML + "noNamespaceSchemaLocation", "QlrFileFormat.xsd")

    FileHeader = ET.SubElement(QlrFile, "FileHeader")
    for k,v in header.items():
        i = ET.SubElement(FileHeader, k )
        i.text = str(v)

    Measurements = ET.SubElement(QlrFile, "Measurements")
    ObjectType =  ET.SubElement(Measurements, "ObjectType")
    ObjectType.text = 'Ticket'

    QlrName =  ET.SubElement(Measurements, "QlrName")
    csv_rows = csv_reader(filename)
    for k,v in enumerate(csv_rows[0] , 1) :
        i = ET.SubElement(QlrName, "N" )
        i.set("i", str(k))
        i.text = v

    QlrData = ET.SubElement(Measurements, "QlrData")
    for k,v in enumerate(csv_rows[1:] , 1) :
        Qlr = ET.SubElement(QlrName, "Qlr" )
        Qlr.set("Dn", str(k))
        for l, w in enumerate(v , 1) :
            i = ET.SubElement(Qlr, "V" )
            i.set("i", str(l) )
            i.text = w
    
    ET.ElementTree(QlrFile).write(os.path.splitext(filename)[0]+".xml", encoding='UTF-8', xml_declaration=True, method='xml', pretty_print = True)

qlr_csv2xml(list(fileheader.keys())[0], fileheader[list(fileheader.keys())[0]])